In [1]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [2]:
import pandas as pd
df = pd.read_csv('data/ratings.csv')

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape)

(785404, 3)


In [4]:
len(df.book_id.unique())

10000

In [5]:
len(df.user_id.unique())

53424

In [6]:
n_books = df["book_id"].nunique()
print(n_books)
n_users = df["user_id"].nunique()
print(n_users)

10000
53424


#### В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [7]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

#### Сначала мы задаём размерность входного слоя. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

##### Делаем то же самое для пользователей:

In [8]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

##### Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

In [9]:
conc = Concatenate()([book_vec, user_vec])

#### Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

In [10]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

##### Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

In [11]:
model2 = Model([user_input, book_input], out)

##### Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam и хорошо известную вам среднеквадратичную ошибку:

In [12]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [13]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 89s 4ms/step - loss: 0.7967
Epoch 2/5
24544/24544 [==============================] - 90s 4ms/step - loss: 0.6893
Epoch 3/5
24544/24544 [==============================] - 89s 4ms/step - loss: 0.6621
Epoch 4/5
24544/24544 [==============================] - 90s 4ms/step - loss: 0.6377
Epoch 5/5
24544/24544 [==============================] - 95s 4ms/step - loss: 0.6171


In [ ]:
model2.evaluate([test.user_id, test.book_id], test.rating)

##### Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [ ]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)